In [3]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import json
from tensorflow.keras.applications import *
from tensorflow.keras import models, layers, optimizers
from sklearn.metrics import classification_report

In [4]:
def plot_images_from_directory(TRAIN_DIR):
    j = 0
    fig = plt.figure(figsize=(15, 15))
    
    for i in os.listdir(TRAIN_DIR):
        print("class", i, ":", os.listdir(TRAIN_DIR + "/" + i)[0])
        img = mpimg.imread(TRAIN_DIR+"/"+i+"/"+os.listdir(TRAIN_DIR+"/"+i)[0])
        fig.add_subplot(2, 3, j+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(img, cmap=plt.cm.binary)
        plt.xlabel(i)
        j += 1
    
    plt.show()

In [5]:
def print_directory_info(DIR):
    for i in os.listdir(DIR):
        files = os.listdir(os.path.join(DIR, i))
        print(f'{i} : {len(files)} files')

In [6]:
def get_early_stopping_settings_string(EARLY_STOPPING):
    if str(EARLY_STOPPING.monitor_op) == "<ufunc 'less'>":
        mode_value = "min"
    elif str(EARLY_STOPPING.monitor_op) == "<ufunc 'greater'>":
        mode_value = "max"
    else:
        mode_value = str(EARLY_STOPPING.monitor_op)

    early_stopping_settings = {
        'monitor': EARLY_STOPPING.monitor,
        'min_delta': EARLY_STOPPING.min_delta,
        'patience': EARLY_STOPPING.patience,
        'mode': mode_value,
        'baseline': EARLY_STOPPING.baseline,
        'restore_best_weights': EARLY_STOPPING.restore_best_weights,
    }
    early_stopping_settings = json.dumps(early_stopping_settings)
    
    return early_stopping_settings


In [7]:
def get_reduce_lr_settings_string(REDUCE_LR):
    reduce_lr_settings = {
        'monitor': REDUCE_LR.monitor,
        'factor': REDUCE_LR.factor,
        'patience': REDUCE_LR.patience,
        'mode': REDUCE_LR.mode,
        'min_delta': REDUCE_LR.min_delta,
        'cooldown': REDUCE_LR.cooldown,
        'min_lr': REDUCE_LR.min_lr
    }
    reduce_lr_settings = json.dumps(reduce_lr_settings)

    
    return reduce_lr_settings

In [8]:
def get_checkpoint_settings_string(MODEL_CHECKPOINT):
    if str(MODEL_CHECKPOINT.monitor_op) == "<ufunc 'less'>":
        mode_value = "min"
    elif str(MODEL_CHECKPOINT.monitor_op) == "<ufunc 'greater'>":
        mode_value = "max"
    else:
        mode_value = str(MODEL_CHECKPOINT.monitor_op)

    checkpoint_settings = {
        'monitor': MODEL_CHECKPOINT.monitor,
        'save_best_only': MODEL_CHECKPOINT.save_best_only,
        'save_weights_only': MODEL_CHECKPOINT.save_weights_only,
        'mode': mode_value,
        'save_freq': MODEL_CHECKPOINT.save_freq
    }
    checkpoint_settings = json.dumps(checkpoint_settings)

    return checkpoint_settings

In [9]:
def get_current_time():
    return datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

In [10]:
def create_saving_path(SAVED_MODEL_DIR, MODEL_BASE_NAME):
    current_time = get_current_time()
    try:
        os.mkdir(SAVED_MODEL_DIR)
    except OSError as error:
        next
    SavingModelPath = SAVED_MODEL_DIR+"/"+MODEL_BASE_NAME+"_"+current_time
    try:
        os.mkdir(SavingModelPath)
    except OSError as error:
        next
        
    return SavingModelPath

In [11]:
def check_base_model(MODEL_BASE_NAME):
    # Get the model class from the global namespace
    try:
        model_class = globals()[MODEL_BASE_NAME]
        conv_base = model_class(
            weights='imagenet',
            include_top=False,
            input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)
        )
        print("Using "+MODEL_BASE_NAME+" as base conv")
        return MODEL_BASE_NAME
    except Exception as error:
        print("error: "+str(error))
        print("")
        return False
    return False


def check_optimizer(OPTIMIZER):
    # Get the model class from the global namespace
    try:
        optimizer=getattr(optimizers, OPTIMIZER)
        print("Using "+OPTIMIZER+" as base optimizers")
        return OPTIMIZER
    except Exception as error:
        print("error: "+str(error))
        print("No "+OPTIMIZER+" optimizers")
        print("")
        return False
    return False

In [12]:
def AUTOTUNE(train_ds, val_ds, test_ds):
    train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)
    val_ds = val_ds.prefetch(tf.data.experimental.AUTOTUNE)
    test_ds = test_ds.prefetch(tf.data.experimental.AUTOTUNE)
    return train_ds, val_ds, test_ds

In [13]:
def plot_acc_loss(history):

    train_acc = history.history['acc']
    val_acc = history.history['val_acc']
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(train_acc)+1)
    
    plt.figure()
    plt.plot(epochs, train_acc, label='Training accuracy')
    plt.plot(epochs, val_acc, label='Validation accuracy', color='red')
    plt.title("Training and Validation Accuracy")
    plt.legend()
    fig = plt.gcf()
    fig.savefig(SavingModelPath+"/"+ "IMG_" +MODEL_BASE_NAME+"_AccPlot.png", dpi=300)
    
    plt.figure()
    plt.plot(epochs, train_loss, label='Training loss')
    plt.plot(epochs, val_loss, label='Validation loss', color='red')
    plt.title("Training and Validation loss")
    plt.legend()
    fig = plt.gcf()
    fig.savefig(SavingModelPath+"/" + "IMG_" + MODEL_BASE_NAME+"_LossPlot.png", dpi=300)
    
    plt.show()
    
    print("Average Validation loss: ", np.mean(val_loss))
    print(f"Average Validation accuracy: {np.mean(val_acc)*100:.2f} %")

In [14]:
def plot_confusion_matrix(test_ds, SavingModelPath, model, MODEL_BASE_NAME):
    # Evaluate the model on the test dataset
    y_true = []
    y_pred = []
    # Record the start time
    start_time = time.time()
    for images, labels in test_ds:
        predictions = model.predict(images, verbose=0)
        predicted_labels = np.argmax(predictions, axis=1)
        y_true.extend(labels.numpy())
        y_pred.extend(predicted_labels)
        
    # Record the end time
    end_time = time.time()
    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    
    
    report_model = classification_report(y_true, y_pred, digits=5)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)
    print(f"Test Accuracy: {accuracy}")

    
    # Calculate confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    # Plot confusion matrix
    plt.figure(figsize=(6, 4))  # Adjust the figure size as needed
    plt.imshow(cm_normalized, interpolation='nearest', cmap=plt.cm.Blues, vmin=0, vmax=1.0)
    plt.title('Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names)
    plt.yticks(tick_marks, class_names)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')

    # Add labels to the confusion matrix plot
    thresh = cm_normalized.max() / 2.0
    for i, j in np.ndindex(cm_normalized.shape):
        plt.text(j, i, format(cm_normalized[i, j], '.2f'), ha='center', va='center', color='white' if cm_normalized[i, j] > thresh else 'black',fontsize=15)

    # Save the confusion matrix plot
    fig = plt.gcf()
    fig.savefig(SavingModelPath + "/" + "IMG_" + MODEL_BASE_NAME + "_Acc_" + str(round(accuracy * 100, 3) ) + ".png", dpi=300)

    plt.show()
    return accuracy, elapsed_time, report_model

In [15]:
def report_to_list(rep, class_names):
    if type(rep) == str:
        lines = rep.split('\n')
        values = [line.split()[0:] for line in lines[0:]]
        values = [item for item in values if item]
        values = [item for item in values if item]

        for i in range(0,len(values)):
            if(i==0):
                values[i].insert(0, '')
            elif(i>len(values)-2-1):
                values[i][0] = ' '.join(values[i][0:2])
                del values[i][1]
            elif(i>len(values)-3-1):
                values[i].insert(1, '')
                values[i].insert(1, '')

            else:
                values[i][0] = class_names[int(values[i][0])]

        return values
    else:
        print("report must be str")
        return rep

# 


# 

# 

# 

In [16]:
#THIS IS OLD SETTING

# EARLY_STOPPING = callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=8, verbose=1, mode="auto", baseline=None, restore_best_weights=True,)
# REDUCE_LR = callbacks.ReduceLROnPlateau(monitor="val_acc", factor=0.6, patience=8, verbose=1, mode="auto", min_delta=0.0001, cooldown=0, min_lr=1e-6,)
# MODEL_CHECKPOINT = callbacks.ModelCheckpoint(
#     os.path.join(SavingModelPath, MODEL_BASE_NAME+"_ModelH5_"+f"{current_time}"+".h5"),
#     monitor = "val_loss", verbose = 1, save_best_only = True, save_weights_only = False, mode = "max", save_freq="epoch")

In [17]:
# THIS IS NEW SETTING

# ### Check default setting at core_function.ipynb
# EARLY_STOPPING = callbacks.EarlyStopping(monitor="val_loss", min_delta=0.001, patience=5, verbose=1, mode="auto", baseline=0.4, restore_best_weights=True,)
# #https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/
# #https://pub.towardsai.net/keras-earlystopping-callback-to-train-the-neural-networks-perfectly-2a3f865148f7

# REDUCE_LR = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.3, patience=3, verbose=1, mode="auto", min_delta=0.0001, cooldown=0, min_lr=1e-6,)
# MODEL_CHECKPOINT = callbacks.ModelCheckpoint(
#     os.path.join(SavingModelPath, "H5Model_"+MODEL_BASE_NAME+"_BestEpoch.h5"),
#     monitor = "val_acc", verbose = 1, save_best_only = True, save_weights_only = False, mode = "auto", save_freq="epoch")